# Quantum associative memory

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

In [2]:
from pyqrack import QrackSimulator, QrackNeuron

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

In [3]:
eta = 1 / 2;
qsim = QrackSimulator(2)
qneuron = QrackNeuron(qsim, [0, 1], 1, eta)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


In [4]:
qneuron.learn_permutation(eta, False)
qsim.x(0)
qneuron.learn_permutation(eta, False)

In [5]:
for perm in range(2):
    qsim.reset_all()
    for i in range(2):
        if (perm >> i) & 1:
            qsim.x(i)
    qneuron.predict()

    comp = qsim.m(1)

    print("Input: ", perm, ", Output: ", comp)

Input:  0 , Output:  1
Input:  1 , Output:  0


In [6]:
input_count = 4
output_count = 4
input_power = 1 << input_count

qsim = QrackSimulator(input_count + output_count)

input_indices = list(range(input_count))

output_layer = []
for i in range(output_count):
    output_layer.append(QrackNeuron(qsim, input_indices, input_count + i))

# Train the network to associate powers of 2 with their log2()
print("Learning (Two's complement)...")
for perm  in range(input_power):
    print("Epoch ", (perm + 1), " out of ", input_power)
    comp = (~perm) + 1
    for i in range(output_count):
        qsim.reset_all()
        for j in range(input_count + output_count):
            if (perm >> j) & 1:
                qsim.x(j)
        output_layer[i].learn_permutation(eta, (comp >> i) & 1)

print("Should associate each input with its two's complement as output...")
for perm in range(input_power):
    qsim.reset_all()
    for j in range(input_count + output_count):
        if (perm >> j) & 1:
            qsim.x(j)
    for i in range(output_count):
        output_layer[i].predict()
    comp = 0
    for i in range(output_count):
        if qsim.m(input_count + i):
            comp |= 1 << i
    print("Input: ", perm, ", Output: ", comp)

Learning (Two's complement)...
Epoch  1  out of  16
Epoch  2  out of  16
Epoch  3  out of  16
Epoch  4  out of  16
Epoch  5  out of  16
Epoch  6  out of  16
Epoch  7  out of  16
Epoch  8  out of  16
Epoch  9  out of  16
Epoch  10  out of  16
Epoch  11  out of  16
Epoch  12  out of  16
Epoch  13  out of  16
Epoch  14  out of  16
Epoch  15  out of  16
Epoch  16  out of  16
Should associate each input with its two's complement as output...
Input:  0 , Output:  0
Input:  1 , Output:  15
Input:  2 , Output:  14
Input:  3 , Output:  13
Input:  4 , Output:  12
Input:  5 , Output:  11
Input:  6 , Output:  10
Input:  7 , Output:  9
Input:  8 , Output:  8
Input:  9 , Output:  7
Input:  10 , Output:  6
Input:  11 , Output:  5
Input:  12 , Output:  4
Input:  13 , Output:  3
Input:  14 , Output:  2
Input:  15 , Output:  1
